In [11]:
# groupby("user").size()

import torch
import lenskit
import pandas as pd
import pickle
import lenskit.crossfold as xf
import numpy as np

In [12]:
ratings = pd.read_csv("../data/ml-20m/ratings.csv")
ratings = ratings.rename(columns={'userId': 'user', 'movieId': 'item_id'})
ratings

,user,item_id,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108
20000260,138493,69644,3.0,1260209457
20000261,138493,70286,5.0,1258126944


In [13]:
item_map = ratings.groupby("item_id").size().reset_index()

In [14]:
keys = item_map["item_id"].values
values = np.arange(item_map.shape[0])
item_dict = dict([(k, v) for (k, v) in zip(keys, values)])

In [15]:
item_indices = [item_dict[item_id] for item_id in ratings["item_id"].values]

In [16]:
ratings["item"] = item_indices

In [17]:
column_titles = ["user", "item", "item_id", "rating", "timestamp"]
ratings = ratings.reindex(columns=column_titles)
ratings["user"] -= 1

In [27]:
ratings

,user,item,item_id,rating,timestamp
0,0,1,2,3.5,1112486027
1,0,28,29,3.5,1112484676
2,0,31,32,3.5,1112484819
3,0,46,47,3.5,1112484727
4,0,49,50,3.5,1112484580
...,...,...,...,...,...
20000258,138492,13754,68954,4.5,1258126920
20000259,138492,13862,69526,4.5,1259865108
20000260,138492,13875,69644,3.0,1260209457
20000261,138492,13993,70286,5.0,1258126944


In [28]:
split1 = xf.partition_users(ratings, 1, xf.SampleFrac(0.3))
train_valtest = next(split1)
train = train_valtest.train
valtest = train_valtest.test

In [29]:
split2 = xf.partition_users(valtest, 1, xf.SampleFrac(0.5))
val_test = next(split2)
val = val_test.train
test = val_test.test

In [30]:
train.groupby("user").size()

user
0         123
1          43
2         131
3          20
4          46
         ... 
138488     27
138489    106
138490     15
138491     57
138492    261
Length: 138493, dtype: int64

In [31]:
val.groupby("user").size()

user
0         26
1          9
2         28
3          4
4         10
          ..
138488     5
138489    23
138490     3
138491    13
138492    56
Length: 138493, dtype: int64

In [32]:
test.groupby("user").size()

user
0         26
1          9
2         28
3          4
4         10
          ..
138488     6
138489    22
138490     4
138491    12
138492    56
Length: 138493, dtype: int64

In [33]:
train.head()

,user,item,item_id,rating,timestamp
0,0,1,2,3.5,1112486027
1,0,28,29,3.5,1112484676
3,0,46,47,3.5,1112484727
4,0,49,50,3.5,1112484580
5,0,110,112,3.5,1094785740


In [34]:
val.head()

,user,item,item_id,rating,timestamp
2,0,31,32,3.5,1112484819
10,0,290,293,4.0,1112484703
12,0,315,318,4.0,1112484798
20,0,907,924,3.5,1094785598
21,0,990,1009,3.5,1112486013


In [35]:
test.head()

,user,item,item_id,rating,timestamp
81,0,2457,2542,4.0,1112485447
68,0,1937,2021,4.0,1112485929
111,0,3902,3996,4.0,1094785727
24,0,1058,1080,3.5,1112485375
43,0,1221,1249,4.0,1112485382


In [36]:
train.to_pickle("../data/ml-20m-split/train.pkl")
val.to_pickle("../data/ml-20m-split/val.pkl")
test.to_pickle("../data/ml-20m-split/test.pkl")

In [37]:
ratings.to_pickle("../data/ml-20m-split/full.pkl")